<a href="https://colab.research.google.com/github/asadbek08/DataScienceMohirdev/blob/main/ML_Ticket_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***This model predicts ticket prices from given different information.***

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

In [ ]:
df_train = pd.read_csv('./train_data.csv')
df_train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [ ]:
df_train.isna().sum()

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [ ]:
X = df_train.drop('price', axis=1)
y = df_train['price']
X['flight'] = X['flight'].str[:3]

In [ ]:
Categorical_part = X.select_dtypes(include=['object']).columns.to_list()
Numerical_part = X.select_dtypes(include=np.number).columns.to_list()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
numerical_transfrom = StandardScaler()
categorical_transform = Pipeline([
    ('one_hot', OneHotEncoder())
])

In [ ]:
processor = ColumnTransformer([
    ('num', numerical_transfrom, Numerical_part),
    ('cat', categorical_transform, Categorical_part)
])

In [ ]:
pipeline = Pipeline([
    ('preprocessor', processor)
])

In [ ]:
X_prepared = pipeline.fit_transform(X)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [ ]:
X_test = pd.read_csv('./test_data.csv')
X_test['flight'] = X_test['flight'].str[:3]
X_test_prepared = pipeline.fit_transform(X_test)

In [ ]:
y_test_predicted = RF_model.predict(X_test_prepared)

In [ ]:
ids = range(1, len(y_test_predicted) + 1)
rf_df = pd.DataFrame({
    'id' : ids,
    'price': y_test_predicted
})
rf_df.to_csv('submission.csv', index=False)
rf_df

,id,price
0,1,51067.73
1,2,55377.61
2,3,22718.52
3,4,3259.24
4,5,6218.84
...,...,...
4995,4996,58357.58
4996,4997,4461.06
4997,4998,50271.68
4998,4999,39919.42
